# Research Notes : Image Histograms
---

This notebook will contain notes on my current research on Zinc Oxide Processing, Structure, and Property.
Here, I will focus on the image histograms subtopic.

## Image histogram variance distribution

### What?

This section discusses properties of variance (_interclass_ and/or intraclass variance distribution in a bimodal, _or similar_, image histogram).
An image histogram is a collection of pixel intensities.
I present notes on image histograms, highlighting comparisons on the distribution of variances in different aspects such as bin count parity, symmetry, peak ratio, and peak position.

### Why?

It is used for Otsu's thresholding method for deteriming an optimal threshold for image segmentation into major groups.

In [ ]:
using Plots, StatsBase; plotly() 

In [ ]:
# interclass variance function
function intervar(hist::FrequencyWeights, thresh::Int)
  ω_k = fweights(hist[1:thresh]).sum / hist.sum
  μ_T = mean(vec(1:length(hist)), hist) / hist.sum
  μ_0 = mean(vec(1:thresh), fweights(hist[1:thresh])) / hist.sum
  μ_1 = (μ_T-(ω_k*μ_0))/(1-ω_k)
    
  return (ω_k*(1-ω_k)) * ((μ_0 - μ_1)^2)
end

### Symmetric Image Histograms

In [ ]:
# construct symmetric bimodal histograms
hist1 = fweights([2, 30, 6, 4, 3, 3, 3, 4, 6, 30, 2])
hist2 = fweights([2, 30, 6, 4, 3, 3, 4, 6, 30, 2])

# indices
x1 = 2:10
x2 = 2:9

# compute respective interclass variance at specified thresh
y1 = [intervar(hist1, i) for i in x1]
y2 = [intervar(hist2, i) for i in x2]

plot(x1, y1)
plot!(x2, y2)

The graph above present the interclass variance plotted against a _test_ threshold for two series.
The first series (in blue) presents interclass variance values for a _symmetric_ image histogram with an __odd__ number of bins.
For this type of series, it is found that there are two possible optimal thresholds.
This is because there are, apparently, two _identical_ ways to divide the histogram into two groups of maximum variance.
Notice that at `thresh = 5|6`, the image histogram can be split into two subsets, $[2, 15, 10, 4, 3]$ and $[3, 3, 4, 10, 15, 2]$.

For the second series (in blue), there is only one possible optimal threshold.
This is because the histogram can be split into _identical_ subsets.
Apparently, the interclass variance of the subsets are maximized when the sum is evenly split into the two groups at `thresh = 5`.
This hypothesis is tested in the next set of graphs.

### Asymmetric Image Histograms

In [ ]:
# construct asymmetric two-peaked image histograms
hist3 = fweights([2, 40, 6, 4, 3, 3, 4, 6, 20, 2])
hist4 = fweights([2, 20, 6, 4, 3, 3, 4, 6, 40, 2])

y3 = [intervar(hist3, i) for i in x2]
y4 = [intervar(hist4, i) for i in x2]

plot(x2, y2)
plot!(x2, y3)
plot!(x2, y4)

In the second graph, a constant sum was distributed over $10$ bins for three histograms.
The first histogram represents a completely symmetric bimodal distrubution.
The second and third represent reflected asymmetric two-peaked distributons.
As shown above, the interclass variance plot for the first histogram (blue) is symmetric with respect to an axis `thresh = 5`.

The two other histograms are asymmetric individually, but since they are reflections of each other, the interclass variance plots are reflections across an axis `thresh = 5`.
Note that the optimal thresholds remain, surprisingly, at the same axis.
The next set of graphs tackles the assumption that for any bimodal distribution, with minimal regard to peak ratio, will have its optimal threshold very near the midpoint of peak bins.

### Midpoint Maximum Sustenance Conjecture

In [ ]:
# construct three asymmeteric two-peaked histograms
hist2 = fweights([2, 20, 2, 2, 20, 2, 2, 2, 2, 2])
hist3 = fweights([2, 20, 2, 2, 2, 20, 2, 2, 2, 2])
hist4 = fweights([2, 20, 2, 2, 2, 2, 20, 2, 2, 2])
hist5 = fweights([2, 20, 2, 2, 2, 2, 2, 20, 2, 2])

y2 = [intervar(hist2, i) for i in x2]
y3 = [intervar(hist3, i) for i in x2]
y4 = [intervar(hist4, i) for i in x2]
y5 = [intervar(hist5, i) for i in x2]

plot(x2, y2)
plot!(x2, y3)
plot!(x2, y4)
plot!(x2, y5)

The graph above shows interclass variance plotted against different threshold values for four different bimodal delta histograms.

It can be easily noted that average interclass variance increases as the distance between delta bins increase.
This shows that the distances between deltas have effects on the __finite difference__ (_derivative_ in the continuous sense) of interclass variances between successive thresholds.

It can also be noted that graph __concavity__ begins to change at the _second_ delta threshold.
Alongside the change in concavity as an effect of the position of the threshold, the __position__ of the optimal threshold also changes.
Through observation of values from the graph, it is found that the optimal threshold value is the midpoint of the delta bins of there are an odd number of intermediate bins, while indeterminate (either of the medians) if there are an even number of intermediate bins.

The probability of occurence of the position of the optimal threshold for odd intermediates can be empirically determined as below.

In [ ]:
# empirically determine probability (after 1000 test cases) of location relative to midpoint
# if N(int bins) is odd, look for the midpoint and test the vicinity
# else if odd, check which is higher, left or right

# each histogram will have 505 elements
# two for prefix bins, and variable lengths for intermediate and suffix bins

## NOTE : there may be extraneous cases wherein the sum of the three counters != 1000
##        suggesting that the conjecture is false.

function prob_extra(peak::Int, dev::Int)
# declare counters
left = 0; right = 0; midpoint = 0
extraneous = 0 # extraneous result counter

# start loop
    for i = 1:500
        prefix = [1, peak]; inter = ones(Int, i); suffix = vcat([peak], ones(Int, 502-i))
        test_hist = fweights(vcat(prefix, inter, suffix))
        if isodd(i)
            # get delta bin midpoint and respective interclass variance at pt
            md = 2 + div(i+1, 2); i_md = intervar(test_hist, md)

            # if either of the surrounding thresholds produce higher interclass variance
            # it is an extraneous case
            if (i_md < intervar(test_hist, md+dev)) | (i_md < intervar(test_hist, md-dev))
                extraneous += 1
            end 
        else
            # check left, right, and one step; add to respective counter
            lft = 2 + div(i, 2); i_lft = intervar(test_hist, lft)
            rt = lft + 1; i_rt = intervar(test_hist, rt)

            # if either of the one-step neighbor bins produce higher interclass variance
            # it is an extraneous case
            if (i_lft < intervar(test_hist, lft-dev)) | (i_rt < intervar(test_hist, rt+dev))
                extraneous += 1
            end
        end
    end
    return extraneous/505
end # function end

delta_cov(peak::Int) = (2*peak)/(503+2*peak)

In [ ]:
x = [50, 100, 500, 1000, 5000, 10000, 20000, 30000, 40000, 42500, 45000, 47500, 50000]

plot(x, [prob_extra(i, 1) for i in x])

Shown above, for low levels of __peak coverage__, the probability that the threshold is not in the middle of the peaks is high.
This suggests, that the conjecture is _false_.
However, it is guaranteed that there is at least a $90\%$ chance for a midpoint threshold to be a the optimal threshold at a peak coverage of $99.44\%$.

In [ ]:
plot([delta_cov(i) for i in x], [prob_extra(i, 1) for i in x])

As above, a sharp decrease in extraneous case percentage begins at a _large coverage_ at about $95\%$.
Since most real cases of bimodal image histograms (grayscale, among others) have a spectrum of possible values, having this amount of coverage is considered __improbable__.

Since this considers only the direct neighbors of the midpoints, we can expect that if we consider a certain margin of error around the midpoint, delta intensity required will be lower to get accurate measures.

In the next cell, sice we have  $505$ cells, at $5\%$ error, we should expect the optimal threshold to be at a cell that whose distance __does not exceed__ $10$ cells from the midpoint of the delta bins. 
We try and plot values for an absolute distance of $4$, $6$, and $10$ cells to see the probabilities of extraneous results. 

### "Extraneous cases" probability as per Midpoint Deviation

In [ ]:
plot([delta_cov(i) for i in x], [prob_extra(i, 4) for i in x])
plot!([delta_cov(i) for i in x], [prob_extra(i, 6) for i in x])
plot!([delta_cov(i) for i in x], [prob_extra(i, 10) for i in x])

Above we see that there is still __much error__ involved in __estimating__ the position of the optimal threshold as a _fixed distance_ from the delta midpoint.

In the next cell, we build a function, similar to `prob_extra(peak::Int, dev::Int)`; instead of asking for a fixed distance from the delta midpoint, we ask a __fixed fraction of the intermediate bins__, `frac <: AbstractFloat`, as deviation.

In [ ]:
## function declaration

function prob_extra(peak::Int, frac::T) where T <: AbstractFloat
# declare counters
left = 0; right = 0; midpoint = 0
extraneous = 0 # extraneous result counter

# start loop
    for i = 1:500
        prefix = [1, peak]; inter = ones(Int, i); suffix = vcat([peak], ones(Int, 502-i))
        test_hist = fweights(vcat(prefix, inter, suffix))
        dev = Int(floor(div(i, 2)*frac))
        if isodd(i)
            # get delta bin midpoint and respective interclass variance at pt
            md = 2 + div(i+1, 2); i_md = intervar(test_hist, md)

            # if either of the surrounding thresholds produce higher interclass variance
            # it is an extraneous case
            if (i_md < intervar(test_hist, md+dev)) | (i_md < intervar(test_hist, md-dev))
                extraneous += 1
            end 
        else
            # check left, right, and one step; add to respective counter
            lft = 2 + div(i, 2); i_lft = intervar(test_hist, lft)
            rt = lft + 1; i_rt = intervar(test_hist, rt)

            # if either of the devstep neighbor bins produce higher interclass variance
            # it is an extraneous case
            if (i_lft < intervar(test_hist, lft-dev)) | (i_rt < intervar(test_hist, rt+dev))
                extraneous += 1
            end
        end
    end
    return extraneous/505
end # function end

Finally, we try the function for medium deviations from the delta midpoint at $10\%$, $20\%$, $30\%$, and $50\%$ of the intermediate bins.

In [ ]:
# try the function for some values
plot([delta_cov(i) for i in x], [prob_extra(i, 0.1) for i in x])
plot!([delta_cov(i) for i in x], [prob_extra(i, 0.2) for i in x])
plot!([delta_cov(i) for i in x], [prob_extra(i, 0.3) for i in x])
plot!([delta_cov(i) for i in x], [prob_extra(i, 0.5) for i in x])

Here we see that, despite the $50\%$ margin of error from the midpoint, there is still a recorded $44\%$ deviation from the midpoint at about $\frac{2}{3}$ coverage.
This shows that the midpoint may __not__ be __a good measure__ of tendency for larger image histograms.

### Non-Delta Image Histograms

The __major difference__ between _non-delta_ from _delta_ image histograms is the variance around the peaks.
In _delta_ image histograms, the variance  of the region around a peak is $0$.
This is not the case for _non-delta_ histograms; variances are non-zero.

In the following sections we explore the consequences of __local variances__ to the location of the optimal threshold.

First, we define a function that creates an image histogram of given local variances around the peaks, given their location.

In [ ]:
# create histogram from peak positions and local variance

### Summary

As shown above, the graph of interclass variance with respect to the threshold is concave throughout the majority of the image histogram.
This presents that there can only be at most two discrete values for the computed optimal threshold.
The graphs also show that the optimal thresholds are always between the peaks in bimodal histograms.
For an image histogram with a small number of bins, the optimal threshold is not far from the midpoint.
As the size grows, however, the probability that it is not near the midpoint also increases.
The problem can be solved through minimizing the variances near the peaks to look similar to combinations of delta functions, but the error from increments is too large to even be considered.